<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/BBC_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/BBC Dataset/bbcsummtrain.csv')

In [ ]:
train

,Unnamed: 0,News,Summary,Category_id,Type,News_lower,news_wo_nltab,news_wo_stop,news_wo_no,news_stem(porter),news_stem(lancast),news_stem(snow),news_stem(reg),news_lemmatize(wordnet),news_wo_punct,news_wo_punct&single,news_tokenized(reg),news_tokenized(sent),news_tokenized(words)
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play with politics\n \n after bub...,online games play with politics. . after bubbl...,"online games play politics. . bubbling time, o...","online games play politics. . bubbling time, o...","onlin game play politics. . bubbl time, onlin ...","onlin gam play politics. . bubbl time, onlin g...","onlin game play politics. . bubbl time, onlin ...","onlin game play politics. . bubbl time, onlin ...","online game play politics. . bubbling time, on...",online game play politics. . bubbling time on...,. bubbling time online game broke onto politi...,[' bubbling time online game broke onto politi...,"[' .', 'bubbling time online game broke onto p...","['bubbling', 'time', 'online', 'game', 'broke'..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' his pm ambition\n \n former c...,hague 'given up' his pm ambition. . former con...,hague 'given up' pm ambition. . former conserv...,hague 'given up' pm ambition. . former conserv...,hagu 'given up' pm ambition. . former conserv ...,hagu 'given up' pm ambition. . form conserv le...,hagu given up pm ambition. . former conserv le...,hagu 'given up' pm ambition. . former conserva...,hague 'given up' pm ambition. . former conserv...,hague given up pm ambition. . former conserv...,. former conservative leader william hague sa...,[' former conservative leader william hague sa...,"[' .', 'former conservative leader william hag...","['former', 'conservative', 'leader', 'william'..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices by average of 24%\n \n marks &...,m&s cuts prices by average of 24%. . marks & s...,m&s cuts prices average 24%. . marks & spencer...,m&s cuts prices average %. . marks & spencer c...,m& cut price averag %. . mark & spencer cut pr...,m&s cut pric av %. . mark & spent cut pric lon...,m&s cut price averag %. . mark & spencer cut p...,m&s cut price averag %. . mark & spencer cut p...,m&s cut price average %. . mark & spencer cut ...,m s cut price average . . mark spencer cut ...,. mark spencer cut price london region averag...,[' mark spencer cut price london region averag...,"[' .', 'mark spencer cut price london region a...","['mark', 'spencer', 'cut', 'price', 'london', ..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident and cautious\n \n zambia's te...,zambia confident and cautious. . zambia's tech...,zambia confident cautious. . zambia's technica...,zambia confident cautious. . zambia's technica...,zambia confid cautious. . zambia' technic dire...,zamb confid cautious. . zambia's techn directo...,zambia confid cautious. . zambia technic direc...,zambia confident cautious. . zambia' technical...,zambia confident cautious. . zambia's technica...,zambia confident cautious. . zambia s technica...,. zambia technical director kalusha bwalya co...,[' zambia technical director kalusha bwalya co...,"[' .', 'zambia technical director kalusha bwal...","['zambia', 'technical', 'director', 'kalusha',..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,federer joins all-time greats. . the last year...,federer joins all-time greats. . last year see...,federer joins all-time greats. . last year see...,feder join all-tim greats. . last year seen on...,fed join all-time greats. . last year seen on ...,feder join all-tim greats. .

In [ ]:
from sklearn import model_selection
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(train['news_wo_punct&single'],train['Category_id'],test_size=0.25)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['news_wo_punct&single'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

NameError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
from sklearn import  naive_bayes, svm
from sklearn.metrics import accuracy_score
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)




SVM Accuracy Score ->  97.07865168539325


In [ ]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  96.85393258426967


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os

In [ ]:
import pandas as pd
testdata = pd.read_csv('/content/drive/MyDrive/BBC Dataset/bbcsummtest.csv')

In [ ]:
testdata

,Unnamed: 0,News,Summary,Category_id,Type
0,1288,Labour battle plan 'hides Blair'\n \n The Tori...,Mr Milburn said the economy would take centre ...,2,politics
1,1170,Tutu's Guantanamo release call\n \n Archbishop...,The South African archbishop said detentions w...,2,politics
2,1388,Holmes starts 2005 with GB events\n \n Kelly H...,Holmes will make her first track appearance on...,3,sport
3,1109,Tory expert denies defeatism\n \n The Conserva...,Mr Howard appointed Mr Crosby as his elections...,2,politics
4,1851,Movie body targets children's PCs\n \n The bod...,The Movie Association for America's (MPAA) fre...,4,tech
...,...,...,...,...,...
440,1117,NHS waiting time target is cut\n \n Hospital w...,"""But let us not forget two thirds of those wai...",2,politics
441,1234,Baron Kinnock makes Lords debut\n \n Former La...,Former Labour leader Neil Kinnock has official...,2,politics
442,1720,Player burn-out worries Robinson\n \n England ...,England coach Andy Robinson says English rugby...,3,sport
443,1230,Blunkett hints at election call\n \n Ex-Home S...,He clarified that he meant he would be in his ...,2,politics


In [ ]:
testdata.drop(testdata.columns[testdata.columns.str.contains('unnamed',case= False)],axis=1,inplace=True)
testdata

,News,Summary,Category_id,Type
0,Labour battle plan 'hides Blair'\n \n The Tori...,Mr Milburn said the economy would take centre ...,2,politics
1,Tutu's Guantanamo release call\n \n Archbishop...,The South African archbishop said detentions w...,2,politics
2,Holmes starts 2005 with GB events\n \n Kelly H...,Holmes will make her first track appearance on...,3,sport
3,Tory expert denies defeatism\n \n The Conserva...,Mr Howard appointed Mr Crosby as his elections...,2,politics
4,Movie body targets children's PCs\n \n The bod...,The Movie Association for America's (MPAA) fre...,4,tech
...,...,...,...,...
440,NHS waiting time target is cut\n \n Hospital w...,"""But let us not forget two thirds of those wai...",2,politics
441,Baron Kinnock makes Lords debut\n \n Former La...,Former Labour leader Neil Kinnock has official...,2,politics
442,Player burn-out worries Robinson\n \n England ...,England coach Andy Robinson says English rugby...,3,sport
443,Blunkett hints at election call\n \n Ex-Home S...,He clarified that he meant he would be in his ...,2,politics


In [ ]:
testdata["News_lower"] = testdata["News"].str.lower()
testdata.head()

,News,Summary,Category_id,Type,News_lower
0,Labour battle plan 'hides Blair'\n \n The Tori...,Mr Milburn said the economy would take centre ...,2,politics,labour battle plan 'hides blair'\n \n the tori...
1,Tutu's Guantanamo release call\n \n Archbishop...,The South African archbishop said detentions w...,2,politics,tutu's guantanamo release call\n \n archbishop...
2,Holmes starts 2005 with GB events\n \n Kelly H...,Holmes will make her first track appearance on...,3,sport,holmes starts 2005 with gb events\n \n kelly h...
3,Tory expert denies defeatism\n \n The Conserva...,Mr Howard appointed Mr Crosby as his elections...,2,politics,tory expert denies defeatism\n \n the conserva...
4,Movie body targets children's PCs\n \n The bod...,The Movie Association for America's (MPAA) fre...,4,tech,movie body targets children's pcs\n \n the bod...


In [ ]:
testdata['news_wo_nltab'] = testdata['News_lower'].replace('\n','.', regex=True)
testdata['news_wo_nltab'] = testdata['news_wo_nltab'].replace('\t',' ', regex=True)
testdata['news_wo_nltab'] = testdata['news_wo_nltab'].replace(r'\.+', ".", regex=True)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
testdata["news_wo_stop"] = testdata["news_wo_nltab"].apply(lambda text: remove_stopwords(text))
#traindata.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
testdata['news_wo_no'] = testdata['news_wo_stop'].str.replace('(\d*\.\d+)|(\d+\.[0-9 ]+)|(\d)', '')

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

testdata["news_lemmatize(wordnet)"] = testdata["news_wo_no"].apply(lambda text: lemmatize_words(text))
#traindata.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import string
PUNCT_TO_REMOVE =  '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    no_punct = ""
    for char in text:
      if char not in PUNCT_TO_REMOVE:
        no_punct = no_punct + char
      else:
        no_punct = no_punct + ' '
    return(no_punct)
testdata["news_wo_punct"] = testdata["news_lemmatize(wordnet)"].apply(lambda text: remove_punctuation(text))
#traindata.head()

In [ ]:
testdata['news_wo_punct'] = testdata['news_wo_punct'].replace(r'\.+', ".", regex=True)

In [ ]:
STOPWORDS = ['a' ,'b', 'c','d','e','f' ,'g' ,'h','i','j','k','l','m','n','o' ,'p' ,'q','r','s','t','u' ,'v' ,'w','x','y','z']
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
testdata["news_wo_punct&single"] = testdata["news_wo_punct"].apply(lambda text: remove_stopwords(text))


In [ ]:
def spl(text):
  return text.split('.',1)[1]
testdata["news_wo_punct&single"]=testdata["news_wo_punct&single"].apply(lambda text: spl(text))
testdata.head()

,News,Summary,Category_id,Type,News_lower,news_wo_nltab,news_wo_stop,news_wo_no,news_lemmatize(wordnet),news_wo_punct,news_wo_punct&single
0,Labour battle plan 'hides Blair'\n \n The Tori...,Mr Milburn said the economy would take centre ...,2,politics,labour battle plan 'hides blair'\n \n the tori...,labour battle plan 'hides blair'. . the tories...,labour battle plan 'hides blair'. . tories acc...,labour battle plan 'hides blair'. . tories acc...,labour battle plan 'hides blair'. . tory accus...,labour battle plan hides blair . . tory accus...,. tory accused tony blair terrified scrutiny ...
1,Tutu's Guantanamo release call\n \n Archbishop...,The South African archbishop said detentions w...,2,politics,tutu's guantanamo release call\n \n archbishop...,tutu's guantanamo release call. . archbishop d...,tutu's guantanamo release call. . archbishop d...,tutu's guantanamo release call. . archbishop d...,tutu's guantanamo release call. . archbishop d...,tutu s guantanamo release call. . archbishop d...,. archbishop desmond tutu called release rema...
2,Holmes starts 2005 with GB events\n \n Kelly H...,Holmes will make her first track appearance on...,3,sport,holmes starts 2005 with gb events\n \n kelly h...,holmes starts 2005 with gb events. . kelly hol...,holmes starts 2005 gb events. . kelly holmes s...,holmes starts gb events. . kelly holmes start...,holmes start gb events. . kelly holmes start s...,holmes start gb events. . kelly holmes start s...,. kelly holmes start series race britain. . h...
3,Tory expert denies defeatism\n \n The Conserva...,Mr Howard appointed Mr Crosby as his elections...,2,politics,tory expert denies defeatism\n \n the conserva...,tory expert denies defeatism. . the conservati...,tory expert denies defeatism. . conservatives'...,tory expert denies defeatism. . conservatives'...,tory expert denies defeatism. . conservatives'...,tory expert denies defeatism. . conservatives ...,. conservatives campaign director denied repo...
4,Movie body targets children's PCs\n \n The bod...,The Movie Association for America's (MPAA) fre...,4,tech,movie body targets children's pcs\n \n the bod...,movie body targets children's pcs. . the body ...,movie body targets children's pcs. . body repr...,movie body targets children's pcs. . body repr...,movie body target children's pcs. . body repre...,movie body target children s pcs. . body repre...,. body represents movie industry released lat...


In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['news_wo_punct&single'])

Test_X_Tfidf = Tfidf_vect.transform(testdata['news_wo_punct&single'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()

Test_Y = Encoder.fit_transform(testdata['Category_id'])

In [ ]:
from sklearn import  naive_bayes, svm
from sklearn.metrics import accuracy_score
#SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
#SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)


SVM Accuracy Score ->  97.75280898876404


In [ ]:
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  97.07865168539325


In [ ]:
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(test_data["sentiment"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

In [ ]:
temp_lsa = svdT.transform(Train_X_Tfidf)
    

In [ ]:
testdata.to_csv('/content/drive/MyDrive/BBC Dataset/bbcsummtest.csv')